In [1]:
%load_ext autoreload
%autoreload 2

# **Imports**

In [2]:
from getModel import blModel, eeModel, eeModel_V0, eeModel_V1, eeModel_V2, eeModel_V3
from trainerv2 import Handler

# training tools
from model_builder.core.metrics import multiclass_accuracy
from torch.nn.functional import cross_entropy

# data tools
import numpy as np
import torch
import pickle

/home/ogech/modulation_classification_using_early_exiting/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **Data**

In [3]:
with open('./Data/RML22.pickle.01A', 'rb') as f:
    data = pickle.load(f, encoding="latin1")

In [4]:
mods = [lst[0] for lst in data.keys()]
snrs = [lst[1] for lst in data.keys()]

In [5]:
len(mods), len(snrs)

(210, 210)

In [6]:
dataset = []
for mod, snr in zip(mods, snrs):
    # print(data[(mod, snr)].shape, mod, snr)
    dataset.extend(data[(mod, snr)])

dataset = np.array(dataset)
dataset.shape  # (210*2000, 2, 128)

(420000, 2, 128)

In [7]:
# one hot encode the labels
num_classes = len(set(mods))
name2label = {name: i for i, name in enumerate(np.unique(mods))}
mods = np.array([name2label[name] for name in mods])
labels = torch.nn.functional.one_hot(
    torch.from_numpy(mods), num_classes=num_classes)
labels = torch.repeat_interleave(labels, 2000, dim=0)

In [8]:
class IQDataset:
    def __init__(self, x, y, transform_x=None, transform_y=None):
        self.x = x
        self.y = y
        self.transform_x = transform_x
        self.transform_y = transform_y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        if self.transform_x:
            x = self.transform_x(x)
        elif isinstance(x, np.ndarray):
            x = torch.from_numpy(x)
        if self.transform_y:
            y = self.transform_y(y)
        elif isinstance(y, np.ndarray):
            y = torch.from_numpy(y)
        return x, y

In [9]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=2, test_size=0.4, random_state=0)
dev_idx, test_idx = next(sss.split(dataset, labels))
train_idx, val_idx = next(sss.split(dataset[dev_idx], labels[dev_idx]))

train_data = IQDataset(dataset[train_idx], labels[train_idx])
val_data = IQDataset(dataset[val_idx], labels[val_idx])
test_data = IQDataset(dataset[test_idx], labels[test_idx])

len(train_data), len(val_data), len(test_data)

(151200, 100800, 168000)

In [10]:
one_x, one_y = train_data[0]
one_x.shape, one_y.shape, one_y.dtype

(torch.Size([2, 128]), torch.Size([10]), torch.int64)

# **Baseline Model**

In [11]:
blNet = blModel()

blNet(one_x)

RuntimeError: running_mean should contain 128 elements not 64

In [ ]:
hdl = Handler(blNet, cross_entropy, nepochs=10,
              sch="cosine", modelName="blModel")
hdl.train(train_data, val_data, bs=64, lr=1e-3, wd=1e-4)

In [ ]:
def loss_func(outputs, target, reduction="mean"):
    out1, out2 = outputs
    return cross_entropy(out1, target, reduction=reduction) + cross_entropy(out2, target, reduction=reduction)